<a href="https://colab.research.google.com/github/antopeza/Final-Project-Brainster/blob/main/Copy_Doc2Vec_notoken_50vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from numpy import random
from numpy import vstack
import matplotlib.pyplot as plt
import os 

import re

import string
from string import punctuation

from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import xgboost as xgb

import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, word_tokenize

import gensim
from gensim.parsing.preprocessing import remove_stopwords

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Embedding, LSTM, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

from imblearn.over_sampling import SMOTE
from collections import Counter

#from scikeras.wrappers import KerasClassifier

import pickle
from pickle import dump
from pickle import load

from imblearn.over_sampling import SMOTE
from collections import Counter

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive 
drive.mount('/drive')

Mounted at /drive


In [4]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [5]:
stops = set(stopwords.words('english'))
#print(stops)

In [6]:
stemmer =  PorterStemmer()
lemmer = WordNetLemmatizer()

In [7]:
#df_raw = pd.read_csv(path)
df_raw = pd.read_csv('/drive/My Drive/MACHINE LEARNING/FINAL PROJECT/train.csv')

In [ ]:
# print('shape of df  = ', df_raw.shape)
# print()
# df_raw.head(2)

shape of df  =  (404290, 6)



,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0


In [ ]:
#df_raw[df_raw.isnull().any(axis=1)]

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0
363362,363362,493340,493341,NaN,My Chinese name is Haichao Yu. What English na...,0


In [8]:
df_raw = df_raw.fillna("")
df_raw.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [9]:
# randomly selected samples from df for the new df to perform operations
df = df_raw.sample(50000)
df = df[['question1', 'question2', 'is_duplicate']]
print(df.shape)

(50000, 3)


In [ ]:
d#f_copy = df.copy()

# Clean Data

In [10]:
def get_wordnet_pos(tag):
    if tag.startswith("N"):
        return wordnet.NOUN
    if tag.startswith("J"):
        return wordnet.ADJ
    if tag.startswith("V"):
        return wordnet.VERB
    if tag.startswith("R"):
        return wordnet.ADV
    return wordnet.NOUN

def clean_sentence(sentence, stopwords = False, lemmatize = False, stem = False):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)  # removing all caracters that are not alpha numeric
    #sentence = re.sub(r'\s{2,}', ' ', sentence
   
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    
    if stopwords:
        sentence = remove_stopwords(sentence)  
        
    if lemmatize:
        nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
        wn_tagged = map(lambda x: (x[0], get_wordnet_pos(x[1])), nltk_tagged)
        res_words = []
        for word, tag in wn_tagged:
            if tag is None:
                res_words.append(word)
            else :
                res_words.append(lemmer.lemmatize(word, tag))
        sentence = " ".join(res_words)
        
    if stem:
        sent_stemmed = ''
        for word in sentence.split():
            sent_stemmed += ' '+ stemmer.stem(word)
        sentence = sent_stemmed
   
    return sentence

def get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False):
    #sents1 = dfq[['question1']]  
    #sents2 = dfq[['question2']]
    cleaned_sentences1 = []
    cleaned_sentences2 = []
    
    for index, row in df.iterrows():
        # print (index, row)
        cleaned1 = clean_sentence(row['question1'], stopwords, lemmatize, stem) 
        cleaned2 = clean_sentence(row['question2'], stopwords, lemmatize, stem)

        cleaned_sentences1.append(cleaned1)
        cleaned_sentences2.append(cleaned2)
    return cleaned_sentences1, cleaned_sentences2

In [11]:
def to_df(q1cleaned, q2cleaned, df):
    '''cleaned sentences to df'''
    X_temp1 = pd.DataFrame()
    X_temp2 = pd.DataFrame()
    X_temp1['q1'] = pd.DataFrame(q1cleaned, index = df.index)
    X_temp2['q2'] = pd.DataFrame(q2cleaned, index = df.index)
    X_temp = pd.concat([X_temp1, X_temp2], axis = 1)
    X_temp['is_duplicate'] = df_raw['is_duplicate']
    return X_temp

#### With Stopwords, lemmatized

In [12]:
# With stopwords lemmatized, not stemmed
q1_sw_lemma, q2_sw_lemma = get_cleaned_senteces(df, stopwords = False, lemmatize = True, stem = False)
assert len(q1_sw_lemma) == len(q2_sw_lemma)
df_sw_lemma = to_df(q1_sw_lemma, q2_sw_lemma, df)
df_sw_lemma.shape

(50000, 3)

#### With Stopwords, stemmed

In [13]:
q1_sw_stem, q2_sw_stem = get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = True)
assert len(q1_sw_stem) == len(q2_sw_stem)
df_sw_stem = to_df(q1_sw_stem, q2_sw_stem, df)
df_sw_stem.shape

(50000, 3)

#### With stopwords, not lemmatized nor stemmed

In [14]:
q1_sw, q2_sw = get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False)
assert len(q1_sw) == len(q2_sw)
df_sw = to_df(q1_sw, q2_sw, df)
df_sw.shape

(50000, 3)

#### Without Stopwords, lemmatized


In [15]:
q1_lemma, q2_lemma = get_cleaned_senteces(df, stopwords = True, lemmatize = True, stem = False)
assert len(q1_lemma) == len(q2_lemma)
df_lemma = to_df(q1_lemma, q2_lemma, df)
df_lemma.shape

(50000, 3)

#### Without Stopwords, stemmed

In [16]:
q1_stem, q2_stem = get_cleaned_senteces(df, stopwords = True, lemmatize = False, stem = True)
assert len(q1_stem) == len(q2_stem)
df_stem = to_df(q1_stem, q2_stem, df)
df_stem.shape

(50000, 3)

#### Without Stopwords, not lemmatized nor stemmed

In [17]:
q1_out, q2_out = get_cleaned_senteces(df, stopwords = True, lemmatize = False, stem = False)
assert len(q1_out) == len(q2_out)
df_out = to_df(q1_out, q2_out, df)
df_out.shape

(50000, 3)

In [ ]:
display ('With stopwords, without lemma and stem', df_sw.iloc[[22]])
display ('With stopwords, lemmatized and without stem', df_sw_lemma.iloc[[22]])
display ('With stopwords, stemmed and without lemma', df_sw_stem.iloc[[22]])
display ('Without stopwords, lemma and stem', df_out.iloc[[22]])
display ('Without stopwords and stem, lemmatized', df_lemma.iloc[[22]])
display ('Without stopwords and lemma, stemmed', df_stem.iloc[[22]])

'With stopwords, without lemma and stem'

,q1,q2,is_duplicate
272872,what does it feel like to be on quora,what does it feel like to post on quora,1


'With stopwords, lemmatized and without stem'

,q1,q2,is_duplicate
272872,what do it feel like to be on quora,what do it feel like to post on quora,1


'With stopwords, stemmed and without lemma'

...

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


'Without stopwords and stem, lemmatized'

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


'Without stopwords and lemma, stemmed'

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


In [ ]:
# print('Value counts:\n', X_sw_train['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_train['is_duplicate'].value_counts()/X_sw_train['is_duplicate'].count()*100)
# X_sw_train['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of train labels', color = 'purple', size = 12)
# plt.show()

In [ ]:
# print('Value counts:\n', X_sw_test['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_test['is_duplicate'].value_counts()/X_sw_test['is_duplicate'].count()*100)
# X_sw_test['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of test labels', color = 'purple', size = 12)
# plt.show()

In [ ]:
# display (X_sw_train.head(2))
# display(X_sw_test.head(2))
# display(df_clean.head(2))

In [ ]:
#X_sw_test[X_sw_test.isnull().any(axis=1)]

### TOKENIZATION

In [ ]:
def tokenization(text):
    tokens = re.split(' ',text)
    return tokens
def token(text):
    text = text.apply(lambda x: tokenization(x))
    return text

In [ ]:
#Tokenize - lemmatized data without stopwords
df_lemma['q1'] = df_lemma['q1'].pipe(token) 
df_lemma['q2'] = df_lemma['q2'].pipe(token) 
df_lemma.head(2)

,q1,q2,is_duplicate
296968,"[indian, lamborghini, usa, legally]","[study, usa, indian, good, idea]",0
236824,"[learn, english]","[learn, english]",0


In [ ]:
# Tokenize - lemmatized data with stopwords
df_sw_lemma['q1'] = df_sw_lemma['q1'].pipe(token) 
df_sw_lemma['q2'] = df_sw_lemma['q2'].pipe(token) 
df_sw_lemma.head(2)

In [ ]:
# Tokenize - Stemmed data with stopwords
df_sw_stem['q1'] = df_sw_stem['q1'].pipe(token) 
df_sw_stem['q2'] = df_sw_stem['q2'].pipe(token) 
df_sw_stem.head(2)

,q1,q2,is_duplicate
11965,"[, what, is, the, list, of, job, titl, at, rsm...","[, which, countri, is, the, best, to, work, in...",0
342404,"[, should, peopl, over, 94, not, be, allow, to...","[, should, peopl, over, 97, not, be, allow, to...",1


In [ ]:
# Tokenize - Stemmed data without stopwords
df_stem['q1'] = df_stem['q1'].pipe(token) 
df_stem['q2'] = df_stem['q2'].pipe(token) 
df_stem.head(2)

,q1,q2,is_duplicate
327246,"[, can, i, regist, my, startup, compani, in, t...","[, can, i, start, a, compani, in, the, us, wit...",1
26328,"[, what, is, a, good, entrylevel, job, in, the...","[, what, are, the, best, entri, level, job, in...",0


In [ ]:
# Tokenize - With stopwords, not lemmatized nor stemmed
df_sw['q1'] = df_sw['q1'].pipe(token) 
df_sw['q2'] = df_sw['q2'].pipe(token) 
df_stem.head(2)

In [ ]:
# Tokenize - Without stopwords, not lemmatized nor stemmed
df_out['q1'] = df_out['q1'].pipe(token) 
df_out['q2'] = df_out['q2'].pipe(token) 
df_out.head(2)

NameError: ignored

In [ ]:
df_sw_lemma.head(2)

,q1,q2,is_duplicate
82361,who invent wash machine,who invent the washing machine,1
12785,be hack a crime when someone do it to earn mon...,what do pakistani girl look for in a guy objec...,0


#### Doc2Vec function

In [18]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def doc2vec(df):
  q1 = [TaggedDocument(d, [i]) for i, d in enumerate(df['q1'])]
  q2 = [TaggedDocument(d, [i]) for i, d in enumerate(df['q2'])]
  return q1, q2

In [19]:
def model_doc2vec(q1, vector, epochs):
  q1 = Doc2Vec(q1, dm = 1, vector_size = vector, window = 3, min_count = 2, epochs = epochs)
  return q1

### Run Doc2Vec Function

In [20]:
#lemmatized, without stopwords
q1_lemma_d2v, q2_lemma_d2v = doc2vec(df_lemma)
assert len(q1_lemma_d2v) == len(q2_lemma_d2v)

In [ ]:
#q1_lemma_d2v

In [21]:
#lemmatized with stopwords
q1_sw_lemma_d2v, q2_sw_lemma_d2v = doc2vec(df_sw_lemma)
assert len(q1_sw_lemma_d2v) == len(q2_sw_lemma_d2v)

In [22]:
#Stemmed without stopwords
q1_stem_d2v, q2_stem_d2v = doc2vec(df_stem)
assert len(q1_stem_d2v) == len(q2_stem_d2v)

In [23]:
#Stemmed with stopwords
q1_sw_stem_d2v, q2_sw_stem_d2v = doc2vec(df_sw_stem)
assert len(q1_sw_stem_d2v) == len(q2_sw_stem_d2v)

In [138]:
#With Stopwords, not lemmatized, not stemmed
q1_sw_d2v, q2_sw_d2v = doc2vec(df_sw)
assert len(q1_sw_d2v) == len(q2_sw_d2v)

In [150]:
#Without stopwords, not lemmatized, not stemmed 
q1_out_d2v, q2_out_d2v = doc2vec(df_out)
assert len(q1_out_d2v) == len(q2_out_d2v)

### Activating Doc2Vec model

In [48]:
#PARAMETER FOR DOC2VEC MODEL 
vector = 200
epochs = 30

In [49]:
#lemmatized, without stopwords
X_q1_lemma_d2v = model_doc2vec(q1_lemma_d2v, vector, epochs)
X_q2_lemma_d2v = model_doc2vec(q2_lemma_d2v, vector, epochs)

In [24]:
df.head(2)    #before normalization

,question1,question2,is_duplicate
46958,How do I get a job if I have a Microsoft certi...,How do Microsoft certifications like MCSE or M...,0
77587,I have to write Article about art gallery how ...,Is writing about art easy?,0


In [23]:
df_lemma.head(2)      #Lemmatized

,q1,q2,is_duplicate
46958,job microsoft certification,microsoft certification like mcse mc aid get t...,0
77587,write article art gallery,write art easy,0


In [22]:
print('q1_lemma_d2v', q1_lemma_d2v[0])
print('doc2vec 50 vectors', X_q1_lemma_d2v[0])

q1_lemma_d2v TaggedDocument(job microsoft certification, [0])
doc2vec 50 vectors [-0.25028104  0.2683925  -0.5079234  -0.09077489 -0.01523149  0.10513179
 -0.23044562  0.05421135  0.07380383 -0.10104693 -0.01661292 -0.06701902
 -0.18589404 -0.21295321  0.00778181  0.0808156  -0.05360462 -0.1482917
  0.11687569 -0.04461858  0.1467118  -0.07293259 -0.02177465  0.0838522
  0.07025379  0.03904675  0.03320546  0.04548879 -0.05606633  0.11805557
  0.19127128 -0.18316105 -0.03924641  0.12809114 -0.22393821 -0.07694057
  0.1813332   0.1209712  -0.10370091  0.03813345 -0.21374252 -0.20262517
 -0.15399083 -0.01977595 -0.07538897 -0.08060285  0.03140114  0.10488901
  0.07532268 -0.00900948]


In [50]:
#lemmatized, with stopwords
X_q1_sw_lemma_d2v = model_doc2vec(q1_sw_lemma_d2v, vector, epochs)
X_q2_sw_lemma_d2v = model_doc2vec(q2_sw_lemma_d2v, vector, epochs)

In [77]:
#Stemmed, without stopwords
X_q1_stem_d2v = model_doc2vec(q1_stem_d2v, vector,  epochs)
X_q2_stem_d2v = model_doc2vec(q2_stem_d2v, vector,  epochs)

In [78]:
#Stemmed, with stopwords
X_q1_sw_stem_d2v = model_doc2vec(q1_sw_stem_d2v, vector, epochs)
X_q2_sw_stem_d2v = model_doc2vec(q2_sw_stem_d2v, vector, epochs)

In [139]:
#With Stopwords, not lemmatized, not stemmed
X_q1_sw_d2v = model_doc2vec(q1_sw_d2v, vector, epochs)
X_q2_sw_d2v = model_doc2vec(q2_sw_d2v, vector, epochs)

In [153]:
#Without stopwords, not lemmatized, not stemmed 
X_q1_out_d2v = model_doc2vec(q1_out_d2v, vector, epochs)
X_q2_out_d2v = model_doc2vec(q2_out_d2v, vector, epochs)


#### Convert to array 

In [51]:
# Lemmatized, without stopwords, not stemmed

size = len(df) - 1
arr_q1_lemma = X_q1_lemma_d2v[0] 

for i in range(size):
  arrT = X_q1_lemma_d2v[i+1]
  arr_q1_lemma = np.vstack([arr_q1_lemma, arrT])

arr_q2_lemma = X_q2_lemma_d2v[0] 

for i in range(size):
  arrT = X_q2_lemma_d2v[i+1]
  arr_q2_lemma = np.vstack([arr_q2_lemma, arrT])

assert arr_q1_lemma.shape == arr_q2_lemma.shape

#arr_q1_lemma = create_arrays(X_q1_lemma_d2v)
#arr_q2_lemma = create_arrays(X_q2_lemma_d2v)

In [52]:
# Lemmatized, with stopwords, not stemmed

size = len(df) - 1
arr_q1_sw_lemma = X_q1_sw_lemma_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_lemma_d2v[i+1]
  arr_q1_sw_lemma = np.vstack([arr_q1_sw_lemma, arrT])

arr_q2_sw_lemma = X_q2_sw_lemma_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_lemma_d2v[i+1]
  arr_q2_sw_lemma = np.vstack([arr_q2_sw_lemma, arrT])

assert arr_q1_sw_lemma.shape == arr_q2_sw_lemma.shape

# arr_q1_sw_lemma = create_arrays(X_q1_sw_lemma_d2v)
# arr_q2_sw_lemma = create_arrays(X_q2_sw_lemma_d2v) 

In [79]:
# Stemmed, with stopwords, not lemmatized

size = len(df) - 1
arr_q1_stem = X_q1_stem_d2v[0] 

for i in range(size):
  arrT = X_q1_stem_d2v[i+1]
  arr_q1_stem = np.vstack([arr_q1_stem, arrT])

arr_q2_stem = X_q2_stem_d2v[0] 

for i in range(size):
  arrT = X_q2_stem_d2v[i+1]
  arr_q2_stem = np.vstack([arr_q2_stem, arrT])

assert arr_q1_stem.shape == arr_q2_stem.shape

# arr_q1_stem = create_arrays(X_q1_stem_d2v)
# arr_q2_stem = create_arrays(X_q2_stem_d2v)

In [80]:
# Stemmed, without stopwords, not lemmatized 

size = len(df) - 1
arr_q1_sw_stem = X_q1_sw_stem_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_stem_d2v[i+1]
  arr_q1_sw_stem = np.vstack([arr_q1_sw_stem, arrT])

arr_q2_sw_stem = X_q2_sw_stem_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_stem_d2v[i+1]
  arr_q2_sw_stem = np.vstack([arr_q2_sw_stem, arrT])

assert arr_q1_sw_stem.shape == arr_q2_sw_stem.shape

# arr_q1_sw_stem = create_arrays(X_q1_sw_stem_d2v)
# arr_q2_sw_stem = create_arrays(X_q2_sw_stem_d2v)

In [156]:
# Without stopwords, not lemmatized, not stemmed
size = len(df) - 1
arr_q1_out = X_q1_out_d2v[0] 

for i in range(size):
  arrT = X_q1_out_d2v[i+1]
  arr_q1_out = np.vstack([arr_q1_out, arrT])

arr_q2_out = X_q2_out_d2v[0] 

for i in range(size):
  arrT = X_q2_out_d2v[i+1]
  arr_q2_out = np.vstack([arr_q2_out, arrT])

assert arr_q1_out.shape == arr_q2_out.shape

In [140]:
# With stopword, not lemmatized, not stemmed

size = len(df) - 1
arr_q1_sw = X_q1_sw_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_d2v[i+1]
  arr_q1_sw = np.vstack([arr_q1_sw, arrT])

arr_q2_sw = X_q2_sw_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_d2v[i+1]
  arr_q2_sw = np.vstack([arr_q2_sw, arrT])

assert arr_q1_sw.shape == arr_q2_sw.shape

### Convert to dataframe 

In [53]:
def df_arr(arr_q1, arr_q2, df):
  temp1 = pd.DataFrame(arr_q1)
  temp2 = pd.DataFrame(arr_q2)
  X_temp = pd.concat([temp1, temp2], axis=1)
  y = df['is_duplicate']
  y = np.array(y)
  y_df = pd.DataFrame(y)
  df_X = X_temp.join(y_df,  lsuffix='_left', rsuffix='_right')
  #X_temp['is_duplicate'] = df['is_duplicate']
  return df_X

In [54]:
# Dataframe - Lemmatized without stopwords 
lemma_df = df_arr(arr_q1_lemma, arr_q2_lemma, df)

In [55]:
# Dataframe - Lemmatized with stopwords 
sw_lemma_df = df_arr(arr_q1_sw_lemma, arr_q2_sw_lemma, df)

In [81]:
# Dataframe - Stemmed without stopwords 
stem_df = df_arr(arr_q1_stem, arr_q2_stem, df)

In [82]:
# Dataframe - Lemmatized with stopwords 
sw_stem_df = df_arr(arr_q1_sw_stem, arr_q2_sw_stem, df)

In [141]:
# Dataframe - With stopwords, not lemmatized, not stemmed
sw_df = df_arr(arr_q1_sw, arr_q2_sw, df)

In [159]:
# Dataframe - Without stopwords, not lemmatized, not stemmed
out_df = df_arr(arr_q1_out, arr_q2_out, df)

In [ ]:
stem_df.head(2)


,0_left,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,0_right
0,-0.007656,-0.017189,-0.058634,0.072262,-0.014387,0.013050,-0.043178,-0.065979,0.055877,-0.036416,...,-0.054352,0.042411,-0.017314,-0.025635,0.013351,-0.065088,0.013830,0.062138,0.054103,0
1,-0.031230,0.023416,0.008874,0.074721,-0.016633,-0.025616,-0.050669,-0.018542,0.036259,-0.002108,...,-0.076787,0.021046,-0.026546,0.046404,-0.064159,0.009849,0.009103,0.022082,0.063962,1


In [30]:
df_lemma.head(2)

,q1,q2,is_duplicate
46958,job microsoft certification,microsoft certification like mcse mc aid get t...,0
77587,write article art gallery,write art easy,0


In [29]:
lemma_df.head(2)

,0_left,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,0_right
0,-0.250281,0.268393,-0.507923,-0.090775,-0.015231,0.105132,-0.230446,0.054211,0.073804,-0.101047,...,0.015447,-0.113064,0.021469,0.100241,-0.08906,-0.092462,0.195167,0.278540,-0.025395,0
1,0.042417,0.109407,0.018437,-0.006771,-0.064517,-0.041608,-0.039554,-0.070556,0.149302,-0.058216,...,0.078449,0.072881,-0.073291,-0.031450,0.07713,0.034200,0.182466,0.055447,0.082686,0


### Split Data

In [56]:
# Lemmatized with stopwords 
y_sw_lemma= sw_lemma_df['0_right']
X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma = train_test_split(sw_lemma_df, y_sw_lemma, stratify = y_sw_lemma, test_size = 0.15, random_state = 42)

In [57]:
# Lemmatized without stopwords 
y_lemma= lemma_df['0_right']
X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma = train_test_split(lemma_df, y_lemma, stratify = y_lemma, test_size = 0.15, random_state = 42)

In [83]:
# Stemmed with stopwords 
y_sw_stem= sw_stem_df['0_right']
X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem = train_test_split(sw_stem_df, y_sw_stem, stratify = y_sw_stem, test_size = 0.15, random_state = 42)

In [84]:
# Stemmed without stopwords 
y_stem= stem_df['0_right']
X_train_stem, X_test_stem, y_train_stem, y_test_stem = train_test_split(stem_df, y_stem, stratify = y_stem, test_size = 0.15, random_state = 42)

In [142]:
# With Stopwords, not lemmatized, not stemmed 
y_sw= sw_df['0_right']
X_train_sw, X_test_sw, y_train_sw, y_test_sw = train_test_split(sw_df, y_sw, stratify = y_sw, test_size = 0.15, random_state = 42)

In [162]:
# Without Stopwords, not lemmatized, not stemmed 
y_out= out_df['0_right']
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(out_df, y_out, stratify = y_out, test_size = 0.15, random_state = 42)

# MODELS

In [ ]:
# Creating DataFrame for Evaluations
result_se = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_se.head()

,Model Name,Accuracy,Precision,Recall,F1_score,Log_loss


In [ ]:
def res(model, acc, precision, recall, f1_score, log_loss):
    result_se.loc[len(result_se)]   = [model, acc, precision, recall, f1_score, log_loss]
    pd.set_option('display.max_rows', None)
    display (result_se)
    return result_se

### Random Forest

In [58]:

def rf_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Random Forest Classifier and model fit'''
    
    #splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)

    rf_clf = RandomForestClassifier(random_state = 0)
    rf_param_grid = { "max_depth": [4, None],
               "min_samples_split": [ 20, 40]
              }
    
    
    rf_search = HalvingGridSearchCV(rf_clf, rf_param_grid, scoring='accuracy', resource='n_estimators', max_resources=5,              
                              random_state=0).fit(X_train, y_train)
    
    rf_model = rf_search.best_estimator_
    
    y_pred = rf_model.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    print (rf_model)
    #print('log loss', rf_loss_bow, '\nacc', rf_acc_bow)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, rf_model

In [85]:
#Random Forest # 
acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma, y_pred_rf_lemma, berf_lemma  = rf_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_rf_lemma = classification_report(y_pred_rf_lemma, y_test_lemma)
matrix_rf_lemma = confusion_matrix(y_pred_rf_lemma, y_test_lemma)
print('Classification Report - RF\n', report_rf_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_lemma)


RandomForestClassifier(min_samples_split=20, n_estimators=3, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.96      0.93      0.95      4867
           1       0.88      0.93      0.91      2633

    accuracy                           0.93      7500
   macro avg       0.92      0.93      0.93      7500
weighted avg       0.93      0.93      0.93      7500


Confusion Matrix - RF
 [[4548  319]
 [ 191 2442]]


In [60]:
#Random Forest # 
acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma, y_pred_rf_lemma, berf_lemma  = rf_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_rf_lemma = classification_report(y_pred_rf_lemma, y_test_lemma)
matrix_rf_lemma = confusion_matrix(y_pred_rf_lemma, y_test_lemma)
print('Classification Report - RF\n', report_rf_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_lemma)


RandomForestClassifier(min_samples_split=20, n_estimators=3, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.96      0.93      0.95      4867
           1       0.88      0.93      0.91      2633

    accuracy                           0.93      7500
   macro avg       0.92      0.93      0.93      7500
weighted avg       0.93      0.93      0.93      7500


Confusion Matrix - RF
 [[4548  319]
 [ 191 2442]]


In [43]:
#Random Forest
acc_rf_sw_lemma, precision_rf_sw_lemma, recall_rf_sw_lemma, f1_score_rf_sw_lemma, log_loss_rf_sw_lemma, y_pred_rf_sw_lemma, berf_sw_lemma  = rf_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)

report_rf_sw_lemma = classification_report(y_pred_rf_sw_lemma, y_test_sw_lemma)
matrix_rf_sw_lemma = confusion_matrix(y_pred_rf_sw_lemma, y_test_sw_lemma)
print('Classification Report - RF\n', report_rf_sw_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw_lemma)
print()


RandomForestClassifier(min_samples_split=20, n_estimators=3, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.96      0.94      0.95      4835
           1       0.89      0.92      0.91      2665

    accuracy                           0.93      7500
   macro avg       0.92      0.93      0.93      7500
weighted avg       0.93      0.93      0.93      7500


Confusion Matrix - RF
 [[4533  302]
 [ 206 2459]]



In [191]:
#Random Forest
acc_rf_sw, precision_rf_sw, recall_rf_sw, f1_score_rf_sw, log_loss_rf_sw, y_pred_rf_sw, berf_sw  = rf_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)

report_rf_sw = classification_report(y_pred_rf_sw, y_test_sw)
matrix_rf_sw = confusion_matrix(y_pred_rf_sw, y_test_sw)
print('Classification Report - RF\n', report_rf_sw)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw)
print()

RandomForestClassifier(n_estimators=3, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4758
           1       0.97      0.99      0.98      2742

    accuracy                           0.99      7500
   macro avg       0.98      0.99      0.98      7500
weighted avg       0.99      0.99      0.99      7500


Confusion Matrix - RF
 [[4688   70]
 [  40 2702]]



In [183]:
# max_depth 10
# min_samples 10
#Random Forest
acc_rf_sw_stem, precision_rf_sw_stem, recall_rf_sw_stem, f1_score_rf_sw_stem, log_loss_rf_sw_stem, y_pred_rf_sw_stem, berf_sw_stem  = rf_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)

report_rf_sw_stem = classification_report(y_pred_rf_sw_stem, y_test_sw_stem)
matrix_rf_sw_stem = confusion_matrix(y_pred_rf_sw_stem, y_test_sw_stem)
print('Classification Report - RF\n', report_rf_sw_stem)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw_stem)
print()

RandomForestClassifier(n_estimators=3, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4741
           1       0.98      0.99      0.98      2759

    accuracy                           0.99      7500
   macro avg       0.99      0.99      0.99      7500
weighted avg       0.99      0.99      0.99      7500


Confusion Matrix - RF
 [[4690   51]
 [  38 2721]]



In [192]:
#Random Forest
acc_rf_stem, precision_rf_stem, recall_rf_stem, f1_score_rf_stem, log_loss_rf_stem, y_pred_rf_stem, berf_stem  = rf_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)

report_rf_stem = classification_report(y_pred_rf_stem, y_test_stem)
matrix_rf_stem = confusion_matrix(y_pred_rf_stem, y_test_stem)
print('Classification Report - RF\n', report_rf_stem)
print()
print('Confusion Matrix - RF\n', matrix_rf_stem)
print()

RandomForestClassifier(min_samples_split=10, n_estimators=3, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4757
           1       0.98      0.99      0.98      2743

    accuracy                           0.99      7500
   macro avg       0.99      0.99      0.99      7500
weighted avg       0.99      0.99      0.99      7500


Confusion Matrix - RF
 [[4697   60]
 [  31 2712]]



In [ ]:
#Random Forest
acc_rf_s, precision_rf_stem, recall_rf_stem, f1_score_rf_stem, log_loss_rf_stem, y_pred_rf_stem, berf_stem  = rf_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)

report_rf_stem = classification_report(y_pred_rf_stem, y_test_stem)
matrix_rf_stem = confusion_matrix(y_pred_rf_stem, y_test_stem)
print('Classification Report - RF\n', report_rf_stem)
print()
print('Confusion Matrix - RF\n', matrix_rf_stem)
print()

## Logistic Regression

In [90]:
def lr_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Logistic Regression Classifier and model fit'''
    
    #splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    #lr_param_grid = {'C':[100, 10]}     
    lr_clf = LogisticRegression(C=0.001, random_state = 0)
   
    #lr_search = HalvingGridSearchCV(lr_clf, lr_param_grid, scoring = 'accuracy')
    lr_clf.fit(X_train, y_train)
    
    #lr_model = lr_search.best_estimator_
    
    y_pred = lr_clf.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    #print (lr_model)
    #print('log loss', rf_loss_bow, '\nacc', rf_acc_bow)
    return accuracy, precision, recall, f1_score, log_loss, y_pred

In [76]:
#Logistic Regression
acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma,  y_pred_lr_lemma = lr_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)
print()


report_lr_lemma = classification_report(y_pred_lr_lemma, y_test_lemma)
matrix_lr_lemma = confusion_matrix(y_pred_lr_lemma, y_test_lemma)
print('Classification Report - Logistic Regression\n', report_lr_lemma)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_lemma)


Classification Report - Logistic Regression
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4739
           1       1.00      1.00      1.00      2761

    accuracy                           1.00      7500
   macro avg       1.00      1.00      1.00      7500
weighted avg       1.00      1.00      1.00      7500


Confusion Matrix - Logistic Regression
 [[4739    0]
 [   0 2761]]


In [72]:
#Logistic Regression  ##200
acc_lr_sw_lemma, precision_lr_sw_lemma, recall_lr_sw_lemma, f1_score_lr_sw_lemma, log_loss_lr_sw_lemma,  y_pred_lr_sw_lemma = lr_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)
print()


report_lr_sw_lemma = classification_report(y_pred_lr_sw_lemma, y_test_sw_lemma)
matrix_lr_sw_lemma = confusion_matrix(y_pred_lr_sw_lemma, y_test_sw_lemma)
print('Classification Report - Logistic Regression\n', report_lr_sw_lemma)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_sw_lemma)


Classification Report - Logistic Regression
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4739
           1       1.00      1.00      1.00      2761

    accuracy                           1.00      7500
   macro avg       1.00      1.00      1.00      7500
weighted avg       1.00      1.00      1.00      7500


Confusion Matrix - Logistic Regression
 [[4739    0]
 [   0 2761]]


In [91]:
#Logistic Regression
acc_lr_stem, precision_lr_stem, recall_lr_stem, f1_score_lr_stem, log_loss_lr_stem,  y_pred_lr_stem = lr_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)
print()


report_lr_stem = classification_report(y_pred_lr_stem, y_test_stem)
matrix_lr_stem = confusion_matrix(y_pred_lr_stem, y_test_stem)
print('Classification Report - Logistic Regression\n', report_lr_stem)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_stem)


Classification Report - Logistic Regression
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4739
           1       1.00      1.00      1.00      2761

    accuracy                           1.00      7500
   macro avg       1.00      1.00      1.00      7500
weighted avg       1.00      1.00      1.00      7500


Confusion Matrix - Logistic Regression
 [[4739    0]
 [   0 2761]]


In [40]:
## SAME AS OTEHRS ##
def lr_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Logistic Regression Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    lr_param_grid = {'C':[100, 10, 1.0, 0.1, 0.01], 'solver':['sag', 'lbfgs']}
    lr_clf = LogisticRegression(penalty = 'l2', random_state = 0)
   
    lr_search = HalvingGridSearchCV(lr_clf, lr_param_grid, cv = splits, factor = 2, scoring = 'accuracy',verbose = 0)
    lr_search.fit(X_train, y_train)
    
    lr_model = lr_search.best_estimator_
    
    y_pred = lr_model.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    print (lr_model)
    #print('log loss', rf_loss_bow, '\nacc', rf_acc_bow)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, lr_model 

# XGB 

In [ ]:
def xgb_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for XGB Classifier and model fit'''
   
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    xgb_param_grid = {'n_estimators': [50, 150, 300], 'max_depth': [4, 6, 8, 10]}
    xgb_clf = xgb.XGBClassifier(objective = 'binary:logistic', n_jobs=-1, random_state = 0, probability = True)
   
    xgb_search = HalvingGridSearchCV(xgb_clf, xgb_param_grid, cv = splits, factor = 2, scoring = 'accuracy', n_jobs=-1,
                                          verbose = 0)
    xgb_search.fit(X_train, y_train, early_stopping_rounds = 3, eval_set=[(X_test, y_test)], verbose = False)
    
    xgb_model = xgb_search.best_params_
    xgb_model_est = xgb_search.best_estimator_
    
    y_pred = xgb_model_est.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'weighted')
    recall = metrics.recall_score(y_test, y_pred, average = 'weighted')
    f1_score = metrics.f1_score(y_test, y_pred, average = 'macro')
    print (xgb_model)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, xgb_model_est  

# Support Vector Classification - SVC

In [44]:
from sklearn.svm import SVC
def svc_clf(X_train, X_test, y_train, y_test):
  clf_svc = svm.SVC(kernel='sigmoid', decision_function_shape='ovr', random_state=None)   #kernel default='rbf', decision_function_shape='ovr', random_state=None


  clf_svc.fit(X_train, y_train)

  y_pred = clf_svc.predict(X_test)
  log_loss = metrics.log_loss(y_test, y_pred)
  accuracy = metrics.accuracy_score(y_test, y_pred)
  precision = metrics.precision_score(y_test, y_pred)
  recall = metrics.recall_score(y_test, y_pred)
  f1_score = metrics.f1_score(y_test, y_pred)
  return accuracy, precision, recall, f1_score, log_loss, y_pred

In [92]:
## 200 vectors ##
# SVC 
acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma = svc_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_svc_lemma = classification_report(y_pred_svc_lemma, y_test_lemma)
matrix_svc_lemma = confusion_matrix(y_pred_svc_lemma, y_test_lemma)

print('Classification Report - SVC\n', report_svc_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_lemma)

{'C': 100, 'gamma': 0.1, 'kernel': 'sigmoid'}


ValueError: ignored

In [65]:
## 200 vectors ##
# SVC 
acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma = svc_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_svc_lemma = classification_report(y_pred_svc_lemma, y_test_lemma)
matrix_svc_lemma = confusion_matrix(y_pred_svc_lemma, y_test_lemma)

print('Classification Report - SVC\n', report_svc_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_lemma)

{'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid'}


ValueError: ignored

In [46]:
## SAME AS OTEHRS   OVERFITTING
def svc_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for XGB Classifier and model fit'''
   
    #splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    
    svc_param_grid = {'C': [0.1, 10, 100], 'gamma': [1, 0.1, 0.01], 'kernel': [ 'sigmoid']}
    svc_clf = SVC(random_state = 0, max_iter = 5, probability = True)
   # gamma='scale'
    svc_search = HalvingGridSearchCV(svc_clf, svc_param_grid, factor = 2, scoring = 'accuracy', n_jobs=-1, verbose = 0)
    svc_search.fit(X_train, y_train)
    
    svc_model = svc_search.best_params_
    svc_model_est = svc_search.best_estimator_
    
    y_pred = svc_model_est.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average='weighted')
    recall = metrics.recall_score(y_test, y_pred, average='weighted')
    f1_score = metrics.f1_score(y_test, y_pred, average='macro')
    print (svc_model)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, svc_model_est  

In [47]:
## 100 vector
acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma, svc_model_lemma = svc_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_svc_lemma = classification_report(y_pred_svc_lemma, y_test_lemma)
matrix_svc_lemma = confusion_matrix(y_pred_svc_lemma, y_test_lemma)

print('Classification Report - SVC\n', report_svc_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_lemma)

{'C': 10, 'gamma': 0.01, 'kernel': 'sigmoid'}
Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4739
           1       1.00      1.00      1.00      2761

    accuracy                           1.00      7500
   macro avg       1.00      1.00      1.00      7500
weighted avg       1.00      1.00      1.00      7500


Confusion Matrix - SVC
 [[4739    0]
 [   0 2761]]


# Linear SVC 

In [93]:
from sklearn.calibration import CalibratedClassifierCV
def lin_svc_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for XGB Classifier and model fit'''
   
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    
    svc_param_grid = {'C': [0.1, 1, 10, 100], 'max_iter': [5000, 10000, 15000]}
    svc_clf = LinearSVC(random_state = 0, penalty = 'l2')
    svc_search = HalvingGridSearchCV(svc_clf, svc_param_grid, cv = splits, factor = 2, scoring = 'accuracy', n_jobs=-1, verbose = 0, 
                                        max_resources = 40, aggressive_elimination = False)
    svc_search.fit(X_train, y_train)
    
    predict_proba_dist = svc_search.decision_function(X_test)
    svc_model = svc_search.best_params_
    svc_model_est = svc_search.best_estimator_
    
    y_pred = svc_model_est.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average='weighted')
    recall = metrics.recall_score(y_test, y_pred, average='weighted')
    f1_score = metrics.f1_score(y_test, y_pred, average='macro')
    print (svc_model)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, svc_model_est, predict_proba_dist  

In [94]:
# Linear SVC 

acc_lsvc_lemma, precision_lsvc_lemma, recall_lsvc_lemma, f1_score_lsvc_lemma, log_loss_lsvc_lemma, y_pred_lsvc_lemma, belsvc_lemma, probab_lemma = lin_svc_clf(X_train_lemma, 
                                                                                                        X_test_lemma, y_train_lemma, y_test_lemma)

report_lsvc_lemma = classification_report(y_pred_lsvc_lemma, y_test_lemma)
matrix_lsvc_lemma = confusion_matrix(y_pred_lsvc_lemma, y_test_lemma)
print('Classification Report - Linear SVC\n', report_lsvc_lemma)
print()
print('Confusion Matrix - Linear SVC\n', matrix_lsvc_lemma)

#result_se = res('Linear SVC Classifier', acc_lsvc_lemma, precision_lsvc_lemma, recall_lsvc_lemma, f1_score_lsvc_lemma, log_loss_lsvc_lemma)

{'C': 100, 'max_iter': 10000}
Classification Report - Linear SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4739
           1       1.00      1.00      1.00      2761

    accuracy                           1.00      7500
   macro avg       1.00      1.00      1.00      7500
weighted avg       1.00      1.00      1.00      7500


Confusion Matrix - Linear SVC
 [[4739    0]
 [   0 2761]]


# Activate model functions

#### On Lemmatized data

In [55]:
#Random Forest
acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma, y_pred_rf_lemma, berf_lemma  = rf_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_rf_lemma = classification_report(y_pred_rf_lemma, y_test_lemma)
matrix_rf_lemma = confusion_matrix(y_pred_rf_lemma, y_test_lemma)
print('Classification Report - RF\n', report_rf_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_lemma)


RandomForestClassifier(min_samples_split=40, n_estimators=3, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.98      0.95      0.97      4880
           1       0.92      0.97      0.94      2620

    accuracy                           0.96      7500
   macro avg       0.95      0.96      0.95      7500
weighted avg       0.96      0.96      0.96      7500


Confusion Matrix - RF
 [[4646  234]
 [  82 2538]]


In [ ]:
# SVC 
acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma = svc_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_svc_lemma = classification_report(y_pred_svc_lemma, y_test_lemma)
matrix_svc_lemma = confusion_matrix(y_pred_svc_lemma, y_test_lemma)

print('Classification Report - SVC\n', report_svc_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_lemma)

Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       475
           1       0.97      1.00      0.99       275

    accuracy                           0.99       750
   macro avg       0.99      0.99      0.99       750
weighted avg       0.99      0.99      0.99       750


Confusion Matrix - SVC
 [[467   8]
 [  0 275]]


In [ ]:
# Linear SVC 

acc_lsvc_lemma, precision_lsvc_lemma, recall_lsvc_lemma, f1_score_lsvc_lemma, log_loss_lsvc_lemma, y_pred_lsvc_lemma, belsvc_lemma, probab_lemma = lin_svc_clf(X_train_lemma, 
                                                                                                        X_test_lemma, y_train_lemma, y_test_lemma)

report_lsvc_lemma = classification_report(y_pred_lsvc_lemma, y_test_lemma)
matrix_lsvc_lemma = confusion_matrix(y_pred_lsvc_lemma, y_test_lemma)
print('Classification Report - Linear SVC\n', report_lsvc_lemma)
print()
print('Confusion Matrix - Linear SVC\n', matrix_lsvc_lemma)

#result_se = res('Linear SVC Classifier', acc_lsvc_lemma, precision_lsvc_lemma, recall_lsvc_lemma, f1_score_lsvc_lemma, log_loss_lsvc_lemma)

{'C': 100, 'max_iter': 10000}
Classification Report - Linear SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       467
           1       1.00      1.00      1.00       283

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - Linear SVC
 [[467   0]
 [  0 283]]


In [ ]:
# Lemmatized, without stopwords   ### 50 vectors 

#Random Forest
acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma, y_pred_rf_lemma, berf_lemma  = rf_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_rf_lemma = classification_report(y_pred_rf_lemma, y_test_lemma)
matrix_rf_lemma = confusion_matrix(y_pred_rf_lemma, y_test_lemma)
print('Classification Report - RF\n', report_rf_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_lemma)

#result_se = res('RandomForest Classifier', acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma)

#Logistic Regression
acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma,  y_pred_lr_lemma, belr_lemma = lr_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)
print()


report_lr_lemma = classification_report(y_pred_lr_lemma, y_test_lemma)
matrix_lr_lemma = confusion_matrix(y_pred_lr_lemma, y_test_lemma)
print('Classification Report - Logistic Regression\n', report_lr_lemma)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_lemma)

#result_se = res('Logistic Regression', acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma)

# XGB 

acc_xgb_lemma, precision_xgb_lemma, recall_xgb_lemma, f1_score_xgb_lemma, log_loss_xgb_lemma, y_pred_xgb_lemma, bexgb_lemma = xgb_clf(X_train_lemma.values, 
                                                                                            X_test_lemma.values, y_train_lemma, y_test_lemma)

report_xgb_lemma = classification_report(y_pred_xgb_lemma, y_test_lemma)
matrix_xgb_lemma = confusion_matrix(y_pred_xgb_lemma, y_test_lemma)
print('Classification Report - XGB\n', report_xgb_lemma)
print()
print('Confusion Matrix - XGB\n', matrix_xgb_lemma)

#result_se = res('XGB Classifier', acc_xgb_lemma, precision_xgb_lemma, recall_xgb_lemma, f1_score_xgb_lemma, log_loss_xgb_lemma)

# SVC 
acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma = svc_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_svc_lemma = classification_report(y_pred_svc_lemma, y_test_lemma)
matrix_svc_lemma = confusion_matrix(y_pred_svc_lemma, y_test_lemma)

print('Classification Report - SVC\n', report_svc_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_lemma)

#result_se = res('SVC Classifier', acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma)

# Linear SVC 

acc_lsvc_lemma, precision_lsvc_lemma, recall_lsvc_lemma, f1_score_lsvc_lemma, log_loss_lsvc_lemma, y_pred_lsvc_lemma, belsvc_lemma, probab_lemma = lin_svc_clf(X_train_lemma, 
                                                                                                        X_test_lemma, y_train_lemma, y_test_lemma)

report_lsvc_lemma = classification_report(y_pred_lsvc_lemma, y_test_lemma)
matrix_lsvc_lemma = confusion_matrix(y_pred_lsvc_lemma, y_test_lemma)
print('Classification Report - Linear SVC\n', report_lsvc_lemma)
print()
print('Confusion Matrix - Linear SVC\n', matrix_lsvc_lemma)

#result_se = res('Linear SVC Classifier', acc_lsvc_lemma, precision_lsvc_lemma, recall_lsvc_lemma, f1_score_lsvc_lemma, log_loss_lsvc_lemma)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=10,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       478
           1       0.96      1.00      0.98       272

    accuracy                           0.99       750
   macro avg       0.98      0.99      0.98       750
weighted avg       0.99      0.99      0.99       750


Confusion Matrix - RF
 [[467  11]
 [  0 272]]
LogisticRegression(C=0.1, random_state=0, solver='sag')

Classification Report - Logistic Regression
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       467
           1       1.00      1.00      1.00       283

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - Logistic Regression
 [[467   0]
 [ 

In [ ]:
# Creating DataFrame for Evaluations
result_lemma = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_lemma.head()

,Model Name,Accuracy,Precision,Recall,F1_score,Log_loss


In [ ]:
#result_lemma = res('RandomForest Classifier', acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma)
#result_lemma = res('Logistic Regression', acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma)
#result_lemma = res('XGB Classifier', acc_xgb_lemma, precision_xgb_lemma, recall_xgb_lemma, f1_score_xgb_lemma, log_loss_xgb_lemma)
#result_lemma = res('SVC Classifier', acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma)
#result_lemma = res('Linear SVC Classifier', acc_lsvc_lemma, precision_lsvc_lemma, recall_lsvc_lemma, f1_score_lsvc_lemma, log_loss_lsvc_lemma)

In [ ]:
# Lemmatized, without stopwords  ### 100 vectors

#Random Forest
acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma, y_pred_rf_lemma, berf_lemma  = rf_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_rf_lemma = classification_report(y_pred_rf_lemma, y_test_lemma)
matrix_rf_lemma = confusion_matrix(y_pred_rf_lemma, y_test_lemma)
print('Classification Report - RF\n', report_rf_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_lemma)

#result_se = res('RandomForest Classifier', acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma)

#Logistic Regression
acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma,  y_pred_lr_lemma, belr_lemma = lr_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)
print()


report_lr_lemma = classification_report(y_pred_lr_lemma, y_test_lemma)
matrix_lr_lemma = confusion_matrix(y_pred_lr_lemma, y_test_lemma)
print('Classification Report - Logistic Regression\n', report_lr_lemma)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_lemma)

#result_se = res('Logistic Regression', acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma)

# XGB 

acc_xgb_lemma, precision_xgb_lemma, recall_xgb_lemma, f1_score_xgb_lemma, log_loss_xgb_lemma, y_pred_xgb_lemma, bexgb_lemma = xgb_clf(X_train_lemma.values, 
                                                                                            X_test_lemma.values, y_train_lemma, y_test_lemma)

report_xgb_lemma = classification_report(y_pred_xgb_lemma, y_test_lemma)
matrix_xgb_lemma = confusion_matrix(y_pred_xgb_lemma, y_test_lemma)
print('Classification Report - XGB\n', report_xgb_lemma)
print()
print('Confusion Matrix - XGB\n', matrix_xgb_lemma)

#result_se = res('XGB Classifier', acc_xgb_lemma, precision_xgb_lemma, recall_xgb_lemma, f1_score_xgb_lemma, log_loss_xgb_lemma)

# SVC 
acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma = svc_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_svc_lemma = classification_report(y_pred_svc_lemma, y_test_lemma)
matrix_svc_lemma = confusion_matrix(y_pred_svc_lemma, y_test_lemma)

print('Classification Report - SVC\n', report_svc_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_lemma)

#result_se = res('SVC Classifier', acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma)

# Linear SVC 

acc_lsvc_lemma, precision_lsvc_lemma, recall_lsvc_lemma, f1_score_lsvc_lemma, log_loss_lsvc_lemma, y_pred_lsvc_lemma, belsvc_lemma, probab_lemma = lin_svc_clf(X_train_lemma, 
                                                                                                        X_test_lemma, y_train_lemma, y_test_lemma)

report_lsvc_lemma = classification_report(y_pred_lsvc_lemma, y_test_lemma)
matrix_lsvc_lemma = confusion_matrix(y_pred_lsvc_lemma, y_test_lemma)
print('Classification Report - Linear SVC\n', report_lsvc_lemma)
print()
print('Confusion Matrix - Linear SVC\n', matrix_lsvc_lemma)

#result_se = res('Linear SVC Classifier', acc_lsvc_lemma, precision_lsvc_lemma, recall_lsvc_lemma, f1_score_lsvc_lemma, log_loss_lsvc_lemma)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=10,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      0.95      0.97       493
           1       0.90      1.00      0.95       257

    accuracy                           0.96       750
   macro avg       0.95      0.97      0.96       750
weighted avg       0.97      0.96      0.96       750


Confusion Matrix - RF
 [[466  27]
 [  1 256]]
LogisticRegression(C=0.1, random_state=0, solver='sag')

Classification Report - Logistic Regression
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       467
           1       1.00      1.00      1.00       283

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - Logistic Regression
 [[467   0]
 [ 

In [ ]:
# Lemmatized, without stopwords  ### 200 vectors

#Random Forest
acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma, y_pred_rf_lemma, berf_lemma  = rf_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_rf_lemma = classification_report(y_pred_rf_lemma, y_test_lemma)
matrix_rf_lemma = confusion_matrix(y_pred_rf_lemma, y_test_lemma)
print('Classification Report - RF\n', report_rf_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_lemma)

#result_se = res('RandomForest Classifier', acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma)

#Logistic Regression
acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma,  y_pred_lr_lemma, belr_lemma = lr_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)
print()


report_lr_lemma = classification_report(y_pred_lr_lemma, y_test_lemma)
matrix_lr_lemma = confusion_matrix(y_pred_lr_lemma, y_test_lemma)
print('Classification Report - Logistic Regression\n', report_lr_lemma)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_lemma)

#result_se = res('Logistic Regression', acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma)

# XGB 

acc_xgb_lemma, precision_xgb_lemma, recall_xgb_lemma, f1_score_xgb_lemma, log_loss_xgb_lemma, y_pred_xgb_lemma, bexgb_lemma = xgb_clf(X_train_lemma.values, 
                                                                                            X_test_lemma.values, y_train_lemma, y_test_lemma)

report_xgb_lemma = classification_report(y_pred_xgb_lemma, y_test_lemma)
matrix_xgb_lemma = confusion_matrix(y_pred_xgb_lemma, y_test_lemma)
print('Classification Report - XGB\n', report_xgb_lemma)
print()
print('Confusion Matrix - XGB\n', matrix_xgb_lemma)

#result_se = res('XGB Classifier', acc_xgb_lemma, precision_xgb_lemma, recall_xgb_lemma, f1_score_xgb_lemma, log_loss_xgb_lemma)

# SVC 
acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma = svc_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_svc_lemma = classification_report(y_pred_svc_lemma, y_test_lemma)
matrix_svc_lemma = confusion_matrix(y_pred_svc_lemma, y_test_lemma)

print('Classification Report - SVC\n', report_svc_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_lemma)

#result_se = res('SVC Classifier', acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma)

# Linear SVC 

acc_lsvc_lemma, precision_lsvc_lemma, recall_lsvc_lemma, f1_score_lsvc_lemma, log_loss_lsvc_lemma, y_pred_lsvc_lemma, belsvc_lemma, probab_lemma = lin_svc_clf(X_train_lemma, 
                                                                                                        X_test_lemma, y_train_lemma, y_test_lemma)

report_lsvc_lemma = classification_report(y_pred_lsvc_lemma, y_test_lemma)
matrix_lsvc_lemma = confusion_matrix(y_pred_lsvc_lemma, y_test_lemma)
print('Classification Report - Linear SVC\n', report_lsvc_lemma)
print()
print('Confusion Matrix - Linear SVC\n', matrix_lsvc_lemma)

#result_se = res('Linear SVC Classifier', acc_lsvc_lemma, precision_lsvc_lemma, recall_lsvc_lemma, f1_score_lsvc_lemma, log_loss_lsvc_lemma)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=10,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      0.93      0.96       503
           1       0.87      1.00      0.93       247

    accuracy                           0.95       750
   macro avg       0.93      0.96      0.94       750
weighted avg       0.96      0.95      0.95       750


Confusion Matrix - RF
 [[466  37]
 [  1 246]]
LogisticRegression(C=0.1, random_state=0, solver='sag')

Classification Report - Logistic Regression
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       467
           1       1.00      1.00      1.00       283

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - Logistic Regression
 [[467   0]
 [ 

## Lemmatized with stopwords

In [ ]:
# Lemmatized, with stopwords

#Random Forest
acc_rf_sw_lemma, precision_rf_sw_lemma, recall_rf_sw_lemma, f1_score_rf_sw_lemma, log_loss_rf_sw_lemma, y_pred_rf_sw_lemma, berf_sw_lemma  = rf_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)

report_rf_sw_lemma = classification_report(y_pred_rf_sw_lemma, y_test_sw_lemma)
matrix_rf_sw_lemma = confusion_matrix(y_pred_rf_sw_lemma, y_test_sw_lemma)
print('Classification Report - RF\n', report_rf_sw_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw_lemma)
print()

#Logistic Regression
lr_acc_sw_lemma, lr_loss_sw_lemma, y_pred_lr_sw_lemma = lr_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)
print()

# SVC 
acc_svc_sw_lemma, precision_svc_sw_lemma, recall_svc_sw_lemma, f1_score_svc_sw_lemma, log_loss_svc_sw_lemma, y_pred_svc_sw_lemma = svc_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)

report_svc_sw_lemma = classification_report(y_pred_svc_sw_lemma, y_test_sw_lemma)
matrix_svc_sw_lemma = confusion_matrix(y_pred_svc_sw_lemma, y_test_sw_lemma)

print('Classification Report - SVC\n', report_svc_sw_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw_lemma)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       467
           1       1.00      1.00      1.00       283

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - RF
 [[467   0]
 [  0 283]]

LogisticRegression(C=0.01, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       469
           1       0.98      0.99      0.99       281

    accuracy                           0.99       750
   macro avg       0.99      0.99      0.99       750
weighted avg       0.99      0.99      0.99       750


Confusion Matrix - SVC
 [[46

In [ ]:
#DATAFRAME RESULT FOR STEMMED DATA WITHOUT STOPWORDS
result_df['random_forest_d2v_sw_lemma'] = f1_score_rf_sw_lemma, log_loss_rf_sw_lemma
result_df['log_regression_d2v_sw_lemma'] = lr_acc_sw_lemma, lr_loss_sw_lemma
result_df['svc_d2v_sw_lemma'] = f1_score_svc_sw_lemma, log_loss_svc_sw_lemma

# With stopwords, not lemmatized nor stemmed

In [ ]:
# With stopwords, not lemmatized, nor stemmed

#Random Forest
acc_rf_sw, precision_rf_sw, recall_rf_sw, f1_score_rf_sw, log_loss_rf_sw, y_pred_rf_sw, berf_sw  = rf_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)

report_rf_sw = classification_report(y_pred_rf_sw, y_test_sw)
matrix_rf_sw = confusion_matrix(y_pred_rf_sw, y_test_sw)
print('Classification Report - RF\n', report_rf_sw)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw)
print()

#Logistic Regression
lr_acc_sw, lr_loss_sw, y_pred_lr_sw = lr_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)
print()

# SVC 
acc_svc_sw, precision_svc_sw, recall_svc_sw, f1_score_svc_sw, log_loss_svc_sw, y_pred_svc_sw = svc_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)

report_svc_sw = classification_report(y_pred_svc_sw, y_test_sw)
matrix_svc_sw = confusion_matrix(y_pred_svc_sw, y_test_sw)

print('Classification Report - SVC\n', report_svc_sw)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       468
           1       1.00      1.00      1.00       282

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - RF
 [[467   1]
 [  0 282]]

LogisticRegression(C=0.1, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       468
           1       0.98      0.98      0.98       282

    accuracy                           0.99       750
   macro avg       0.98      0.98      0.98       750
weighted avg       0.99      0.99      0.99       750


Confusion Matrix - SVC
 [[462

In [ ]:
#DATAFRAME RESULT FOR STEMMED DATA WITHOUT STOPWORDS
result_df['random_forest_d2v_sw'] = f1_score_rf_sw, log_loss_rf_sw
result_df['log_regression_d2v_sw'] = lr_acc_sw, lr_loss_sw
result_df['svc_d2v_sw'] = f1_score_svc_sw, log_loss_svc_sw

### Without stopwords, not lemmatized nor stemmed

In [ ]:
# Without stopwords, not lemmatized, nor stemmed

#Random Forest
acc_rf_out, precision_rf_out, recall_rf_out, f1_score_rf_out, log_loss_rf_out, y_pred_rf_out, berf_out  = rf_clf(X_train_out, X_test_out, y_train_out, y_test_out)

report_rf_out = classification_report(y_pred_rf_out, y_test_out)
matrix_rf_out = confusion_matrix(y_pred_rf_out, y_test_out)
print('Classification Report - RF\n', report_rf_out)
print()
print('Confusion Matrix - RF\n', matrix_rf_out)
print()

#Logistic Regression
lr_acc_out, lr_loss_out, y_pred_lr_out = lr_clf(X_train_out, X_test_out, y_train_out, y_test_out)
print()

# SVC 
acc_svc_out, precision_svc_out, recall_svc_out, f1_score_svc_out, log_loss_svc_out, y_pred_svc_out = svc_clf(X_train_out, X_test_out, y_train_out, y_test_out)

report_svc_out = classification_report(y_pred_svc_out, y_test_out)
matrix_svc_out = confusion_matrix(y_pred_svc_out, y_test_out)

print('Classification Report - SVC\n', report_svc_out)
print()
print('Confusion Matrix - SVC\n', matrix_svc_out)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       468
           1       1.00      1.00      1.00       282

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - RF
 [[467   1]
 [  0 282]]

LogisticRegression(C=0.01, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       0.98      0.97      0.98       470
           1       0.96      0.97      0.96       280

    accuracy                           0.97       750
   macro avg       0.97      0.97      0.97       750
weighted avg       0.97      0.97      0.97       750


Confusion Matrix - SVC
 [[45

In [ ]:
#DATAFRAME RESULT FOR STEMMED DATA WITHOUT STOPWORDS
result_df['random_forest_d2v_out'] = f1_score_rf_out, log_loss_rf_out
result_df['log_regression_d2v_out'] = lr_acc_out, lr_loss_out
result_df['svc_d2v_out'] = f1_score_svc_out, log_loss_svc_out

# Stemmed data with stopwords

In [ ]:
# Stemmed with stopwords

#Random Forest
acc_rf_sw_stem, precision_rf_sw_stem, recall_rf_sw_stem, f1_score_rf_sw_stem, log_loss_rf_sw_stem, y_pred_rf_sw_stem, berf_sw_stem  = rf_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)

report_rf_sw_stem = classification_report(y_pred_rf_sw_stem, y_test_sw_stem)
matrix_rf_sw_stem = confusion_matrix(y_pred_rf_sw_stem, y_test_sw_stem)
print('Classification Report - RF\n', report_rf_sw_stem)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw_stem)
print()

#Logistic Regression
lr_acc_sw_stem, lr_loss_sw_stem, y_pred_lr_sw_stem = lr_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)
print()

# SVC 
acc_svc_sw_stem, precision_svc_sw_stem, recall_svc_sw_stem, f1_score_svc_sw_stem, log_loss_svc_sw_stem, y_pred_svc_sw_stem = svc_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)

report_svc_sw_stem = classification_report(y_pred_svc_sw_stem, y_test_sw_stem)
matrix_svc_sw_stem = confusion_matrix(y_pred_svc_sw_stem, y_test_sw_stem)

print('Classification Report - SVC\n', report_svc_sw_stem)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw_stem)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       468
           1       1.00      1.00      1.00       282

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - RF
 [[467   1]
 [  0 282]]

LogisticRegression(C=0.1, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       468
           1       0.99      1.00      0.99       282

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - SVC
 [[466

In [ ]:
#DATAFRAME RESULT FOR STEMMED DATA WITHOUT STOPWORDS
result_df['random_forest_d2v_sw_stemmed'] = f1_score_rf_sw_stem, log_loss_rf_sw_stem
result_df['log_regression_d2v_sw_stemmed'] = lr_acc_sw_stem, lr_loss_sw_stem
result_df['svc_d2v_sw_stem'] = f1_score_svc_sw_stem, log_loss_svc_sw_stem

#result_df.drop(['rf_d2v_sw_stem', 'lr_d2v_sw_stem', 'svc_d2v_sw_stem'], axis=1)

# Stemmed without stopwords


In [ ]:
# Stemmed with stopwords

#Random Forest
acc_rf_stem, precision_rf_stem, recall_rf_stem, f1_score_rf_stem, log_loss_rf_stem, y_pred_rf_stem, berf_stem  = rf_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)

report_rf_stem = classification_report(y_pred_rf_stem, y_test_stem)
matrix_rf_stem = confusion_matrix(y_pred_rf_stem, y_test_stem)
print('Classification Report - RF\n', report_rf_stem)
print()
print('Confusion Matrix - RF\n', matrix_rf_stem)
print()

#Logistic Regression
lr_acc_stem, lr_loss_stem, y_pred_lr_stem = lr_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)
print()

# SVC 
acc_svc_stem, precision_svc_stem, recall_svc_stem, f1_score_svc_stem, log_loss_svc_stem, y_pred_svc_stem, besvc_stem, probab_stem = svc_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)

report_svc_stem = classification_report(y_pred_svc_stem, y_test_stem)
matrix_svc_stem = confusion_matrix(y_pred_svc_stem, y_test_stem)

print('Classification Report - SVC\n', report_svc_stem)
print()
print('Confusion Matrix - SVC\n', matrix_svc_stem)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       467
           1       1.00      1.00      1.00       283

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - RF
 [[467   0]
 [  0 283]]

LogisticRegression(C=0.1, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       0.99      0.97      0.98       475
           1       0.95      0.98      0.97       275

    accuracy                           0.98       750
   macro avg       0.97      0.98      0.97       750
weighted avg       0.98      0.98      0.98       750


Confusion Matrix - SVC
 [[462

In [ ]:
result_df = pd.DataFrame(index = ['accuracy', 'loss'])

In [ ]:
#DATAFRAME RESULT FOR STEMMED DATA WITHOUT STOPWORDS
result_df['random_forest_d2v_stemmed'] = f1_score_rf_stem, log_loss_rf_stem
result_df['log_regression_d2v_stemmed'] = lr_acc_stem, lr_loss_stem
result_df['svc_d2v_stem'] = f1_score_svc_stem, log_loss_svc_stem

#result_df.drop(['rf_d2v_sw_stem', 'lr_d2v_sw_stem', 'svc_d2v_sw_stem'], axis=1)

In [ ]:
result_df

,random_forest_d2v_lemma,log_regression_d2v_lemma,svc_d2v_lemma,random_forest_d2v_sw_lemma,log_regression_d2v_sw_lemma,svc_d2v_sw_lemma,random_forest_d2v_sw,log_regression_d2v_sw,svc_d2v_sw,random_forest_d2v_out,log_regression_d2v_out,svc_d2v_out,random_forest_d2v_sw_stemmed,log_regression_d2v_sw_stemmed,svc_d2v_sw_stem,random_forest_d2v_stemmed,log_regression_d2v_stemmed,svc_d2v_stem
accuracy,0.998230,1.000000e+00,0.959002,1.000000e+00,1.000000e+00,0.985816,0.998230,1.000000e+00,0.980531,0.998230,1.000000e+00,0.962700,0.998230,1.000000e+00,0.994690,1.000000e+00,1.000000e+00,0.967742
loss,0.046052,9.992007e-16,1.059199,9.992007e-16,9.992007e-16,0.368417,0.046052,9.992007e-16,0.506574,0.046052,9.992007e-16,0.967095,0.046052,9.992007e-16,0.138156,9.992007e-16,9.992007e-16,0.828936


# Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#cosine = cosine_similarity(X_train_lemma, y_train_lemma, dense_output=True)

In [ ]:
cosine_similarity(arr_q1_lemma , arr_q2_lemma, dense_output=True)

array([[0.77353704, 0.6493094 , 0.7844405 , ..., 0.61083734, 0.6324987 ,
        0.8206404 ],
       [0.33382827, 0.8227184 , 0.5351572 , ..., 0.5696451 , 0.4045978 ,
        0.5432366 ],
       [0.57329106, 0.65007234, 0.86115694, ..., 0.77194357, 0.52491057,
        0.7496874 ],
       ...,
       [0.5479884 , 0.5654626 , 0.74211514, ..., 0.8167627 , 0.58999306,
        0.8312438 ],
       [0.63509995, 0.54801077, 0.78231716, ..., 0.63727254, 0.88014674,
        0.81301326],
       [0.6825827 , 0.6387254 , 0.84284997, ..., 0.6468858 , 0.6542733 ,
        0.8440794 ]], dtype=float32)

In [ ]:
cosine_similarity(arr_q1_lemma , arr_q2_lemma)
cosine_similarity(arr_q1_sw_lemma , arr_q2_sw_lemma)
cosine_similarity(arr_q1_stem , arr_q2_stem)
cosine_similarity(arr_q1_sw_stem , arr_q2_sw_stem)
cosine_similarity(arr_q1_sw , arr_q2_sw)
cosine_similarity(arr_q1_out , arr_q2_out)

# Doc2Vec and similarity

In [ ]:
model_doc2vec = Doc2Vec(vector_size =20, window=2, min_count=1, epochs=50)

In [ ]:
model_doc2vec.build_vocab(X_train_tagged)

In [ ]:
model_doc2vec.train(X_train_tagged, total_examples=model_doc2vec.corpus_count, epochs=model_doc2vec.epochs)

In [ ]:
print(model_doc2vec.infer_vector(X_sw_test))

[ 0.00553507 -0.02057612 -0.00601494  0.01590716  0.00477619 -0.00957217
 -0.01354616  0.01898067 -0.00717559  0.00853975  0.00150559 -0.01450056
 -0.00628278 -0.02179538 -0.01993607 -0.02084691  0.00312932 -0.0190215
 -0.00581192  0.01894295]


In [ ]:
#TEST THE MODEL ON X_sw_test

test = model_doc2vec.infer_vector(X_sw_test)
model_doc2vec.docvecs.most_similar(positive = [test])
#positive = List of sentences that contribute positively.


[(1607, 0.3674626350402832),
 (1454, 0.3508557975292206),
 (1876, 0.3459669351577759),
 (2316, 0.3180397152900696),
 (1868, 0.31006860733032227),
 (389, 0.3054264485836029),
 (1438, 0.28425800800323486),
 (1760, 0.272051066160202),
 (2109, 0.2675776481628418),
 (2054, 0.2670101821422577)]